Generate sample data

In [1]:
import random

# Lista de posibles valores para cada campo
nombres = ["Juan", "Ana", "Pedro", "Maria", "Carlos", "Sofia"]
apellidos = ["Garcia", "Rodriguez", "Martinez", "Lopez", "Perez", "Gomez"]
tipos_propiedad = ["Casa", "Departamento", "Terreno", "Local comercial"]
cantidades = [100000, 200000, 300000, 400000, 500000]
plazos = [5, 10, 15, 20, 25, 30]


# Función para generar un registro aleatorio
def generar_registro():
    nombre = random.choice(nombres)
    apellido = random.choice(apellidos)
    tipo_propiedad = random.choice(tipos_propiedad)
    cantidad = random.choice(cantidades)
    plazo = random.choice(plazos)
    tasa = round(random.uniform(5, 10), 2)
    return [nombre, apellido, tipo_propiedad, cantidad, plazo, tasa]


# Generar 100 registros
registros = []
for i in range(100):
    registro = generar_registro()
    registros.append(registro)

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Exercise01').getOrCreate()

Ejercicio 1

Crear un DataFrame a partir de los registros y crea una vista temporal

In [4]:
df = spark.createDataFrame(registros, ["nombre", "apellido", "tipo_propiedad", "cantidad", "plazo", "tasa"])
df.createOrReplaceTempView("hipotecas")
df.show()

+------+---------+---------------+--------+-----+----+
|nombre| apellido| tipo_propiedad|cantidad|plazo|tasa|
+------+---------+---------------+--------+-----+----+
|   Ana|   Garcia|   Departamento|  400000|   30|9.99|
|Carlos|Rodriguez|   Departamento|  200000|   10|9.32|
|   Ana| Martinez|           Casa|  200000|   20|8.51|
|   Ana|    Lopez|Local comercial|  100000|   10|5.63|
|  Juan|    Gomez|           Casa|  100000|   15|9.39|
|   Ana|   Garcia|        Terreno|  100000|    5|7.72|
|  Juan|    Gomez|Local comercial|  200000|    5|5.01|
|  Juan|    Lopez|Local comercial|  500000|   10|5.89|
|   Ana|    Lopez|        Terreno|  100000|    5| 7.5|
|  Juan|    Perez|           Casa|  400000|   30|5.41|
| Pedro|Rodriguez|        Terreno|  300000|   25| 5.2|
| Maria|    Perez|           Casa|  400000|   10|5.94|
|Carlos|   Garcia|           Casa|  300000|   25|8.56|
|Carlos|    Gomez|        Terreno|  400000|    5| 5.4|
|  Juan|    Lopez|           Casa|  400000|   25|7.13|
|   Ana|  

Ejercicio 2
Seleccionar los registros de las personas que han tomado una hipoteca mayor de 300.000 €

In [5]:
df.filter(df.cantidad > 300000).show()

+------+---------+---------------+--------+-----+----+
|nombre| apellido| tipo_propiedad|cantidad|plazo|tasa|
+------+---------+---------------+--------+-----+----+
|   Ana|   Garcia|   Departamento|  400000|   30|9.99|
|  Juan|    Lopez|Local comercial|  500000|   10|5.89|
|  Juan|    Perez|           Casa|  400000|   30|5.41|
| Maria|    Perez|           Casa|  400000|   10|5.94|
|Carlos|    Gomez|        Terreno|  400000|    5| 5.4|
|  Juan|    Lopez|           Casa|  400000|   25|7.13|
|Carlos| Martinez|Local comercial|  400000|   10|5.02|
| Maria|Rodriguez|   Departamento|  500000|   15|6.64|
| Maria|    Perez|           Casa|  500000|   25|9.16|
|  Juan|Rodriguez|           Casa|  500000|   10|8.73|
| Sofia|Rodriguez|Local comercial|  400000|    5| 6.3|
| Pedro|Rodriguez|        Terreno|  500000|   20|7.53|
| Sofia| Martinez|Local comercial|  400000|   30|8.07|
|Carlos|    Gomez|           Casa|  500000|    5|5.82|
|Carlos|    Perez|           Casa|  400000|   25|9.11|
| Pedro|Ro

Ejercicio 3

Contar cuántos registros hay para cada tipo de propiedad:

In [6]:
df.groupBy("tipo_propiedad").count().show()

+---------------+-----+
| tipo_propiedad|count|
+---------------+-----+
|   Departamento|   24|
|           Casa|   26|
|Local comercial|   27|
|        Terreno|   23|
+---------------+-----+


Ejercicio 4

Obtener la cantidad total de dinero prestado en hipotecas

In [7]:
df.agg({"cantidad": "sum"}).show()

+-------------+
|sum(cantidad)|
+-------------+
|     31900000|
+-------------+


Ejercicio 5

Obtener la tasa promedio de interés para cada plazo

In [8]:
df.groupby("tipo_propiedad").agg({"tasa": "avg"}).show()

+---------------+-----------------+
| tipo_propiedad|        avg(tasa)|
+---------------+-----------------+
|   Departamento|7.642499999999998|
|           Casa|7.305384615384615|
|Local comercial| 7.48925925925926|
|        Terreno|7.331304347826088|
+---------------+-----------------+


Ejercicio 6
Seleccionar los registros de las personas que han tomado una hipoteca por un monto mayor a $300,000 y cuyo plazo sea mayor a 15 años

In [9]:
df.filter((df.cantidad > 300000) & (df.plazo > 15)).show()

+------+---------+---------------+--------+-----+----+
|nombre| apellido| tipo_propiedad|cantidad|plazo|tasa|
+------+---------+---------------+--------+-----+----+
|   Ana|   Garcia|   Departamento|  400000|   30|9.99|
|  Juan|    Perez|           Casa|  400000|   30|5.41|
|  Juan|    Lopez|           Casa|  400000|   25|7.13|
| Maria|    Perez|           Casa|  500000|   25|9.16|
| Pedro|Rodriguez|        Terreno|  500000|   20|7.53|
| Sofia| Martinez|Local comercial|  400000|   30|8.07|
|Carlos|    Perez|           Casa|  400000|   25|9.11|
| Pedro|Rodriguez|        Terreno|  400000|   30| 6.8|
| Pedro|    Lopez|   Departamento|  500000|   25|5.09|
| Sofia|Rodriguez|   Departamento|  500000|   20|9.89|
| Maria| Martinez|           Casa|  400000|   30|6.39|
|Carlos|   Garcia|           Casa|  500000|   30| 8.0|
|   Ana|    Gomez|           Casa|  500000|   20|5.87|
| Maria|    Gomez|           Casa|  500000|   25|5.62|
|  Juan|    Lopez|        Terreno|  400000|   30|5.26|
|Carlos| M